# CoG paper experiments

## Imports and setup

In [ ]:
import warnings
from typing import Any, Dict, List, Optional, Tuple, Union

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch as th
from pcgsepy.config import (CS_MAX_AGE, EPSILON_F, MAX_X_SIZE, MAX_Y_SIZE,
                            MAX_Z_SIZE, N_GENS, N_ITERATIONS, N_RETRIES,
                            N_RUNS, POP_SIZE)
from pcgsepy.evo.fitness import (Fitness, box_filling_fitness,
                                 func_blocks_fitness, mame_fitness,
                                 mami_fitness)
from pcgsepy.evo.genops import expander
from pcgsepy.fi2pop.utils import (GaussianEstimator, create_new_pool,
                                  prepare_dataset, reduce_population,
                                  subdivide_solutions)
from pcgsepy.lsystem.constraints import ConstraintLevel
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.lsystem.solution import CandidateSolution
from pcgsepy.mapelites.bandit import Bandit, EpsilonGreedyAgent
from pcgsepy.mapelites.behaviors import BehaviorCharacterization, mame, mami
from pcgsepy.mapelites.buffer import (Buffer, EmptyBufferException, max_merge,
                                      mean_merge, min_merge)
from pcgsepy.mapelites.emitters import OptimisingEmitter, OptimisingEmitterV2
from pcgsepy.mapelites.map import MAPElites, coverage_reward, fitness_reward
from pcgsepy.nn.estimators import (MLPEstimator, QuantileEstimator,
                                   train_estimator)
from pcgsepy.setup_utils import get_default_lsystem, setup_matplotlib
from scipy.stats import bootstrap, sem
from tqdm.notebook import trange

warnings.filterwarnings("ignore")

In [ ]:
setup_matplotlib(larger_fonts=False)

used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

lsystem = get_default_lsystem(used_ll_blocks=used_ll_blocks)

## Plotting and saving utils

In [ ]:
def plot_mean_and_top_feasible(ffs,
                               exp_name,
                               dir_name):
    ftfs = np.max(ffs, axis=2)
    fmfs = np.mean(ffs, axis=2)

    ftm = np.mean(ftfs, axis=0)
    fts = np.std(ftfs, axis=0)

    fmm = np.mean(fmfs, axis=0)
    fms = np.std(fmfs, axis=0)

    plt.grid()
    plt.plot(range(len(ftm)), ftm, label=f'Top fitness', c='blue', lw=2)
    plt.fill_between(range(len(fts)), (ftm - (.5 * fts)), (ftm + (.5 * fts)), color='blue', alpha=0.2)
    plt.fill_between(range(len(fts)), (ftm - fts), (ftm + fts), color='blue', alpha=0.1)
    plt.plot(range(len(fmm)), fmm, label=f'Mean fitness', c='darkgreen', lw=2)
    plt.fill_between(range(len(fms)), (fmm - (.5 * fms)), (fmm + (.5 * fms)), color='darkgreen', alpha=0.2)
    plt.fill_between(range(len(fms)), (fmm - fms), (fmm + fms), color='darkgreen', alpha=0.1)
    plt.legend(loc='lower right')
    plt.title(f'Avg. FPop fitness ({N_RUNS} runs)')
    plt.ylabel('Fitness')
    plt.xlabel('Generations')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.savefig(f'results/{dir_name}/{exp_name}-fpop-avgf.png', transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_mean_and_top_infeasible(ifs,
                                 exp_name,
                                 dir_name):
    itfs = np.max(ifs, axis=2)
    imfs = np.mean(ifs, axis=2)

    itm = np.mean(itfs, axis=0)
    its = np.std(itfs, axis=0)

    imm = np.mean(imfs, axis=0)
    ims = np.std(imfs, axis=0)

    plt.grid()
    plt.plot(range(len(itm)), itm, label=f'Top fitness', c='blue', lw=2)
    plt.fill_between(range(len(its)), (itm - (.5 * its)), (itm + (.5 * its)), color='blue', alpha=0.2)
    plt.fill_between(range(len(its)), (itm - its), (itm + its), color='blue', alpha=0.1)
    plt.plot(range(len(imm)), imm, label=f'Mean fitness', c='darkgreen', lw=2)
    plt.fill_between(range(len(ims)), (imm - (.5 * ims)), (imm + (.5 * ims)), color='darkgreen', alpha=0.2)
    plt.fill_between(range(len(ims)), (imm - ims), (imm + ims), color='darkgreen', alpha=0.1)
    plt.legend(loc='upper right')
    plt.title(f'Avg. IPop fitness ({N_RUNS} runs)')
    plt.ylabel('Fitness')
    plt.xlabel('Generations')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.savefig(f'results/{dir_name}/{exp_name}-ipop-avgf.png', transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_coverage(coverages_hist,
                  exp_name,
                  title: str,
                  filename: str,
                  dir_name):
    mean_coverage = np.mean(coverages_hist, axis=0)
    std_coverage = np.std(coverages_hist, axis=0)
    
    plt.grid()
    plt.plot(range(len(mean_coverage)), mean_coverage, label=f'Avg. coverage', c='blue', lw=2)
    plt.fill_between(range(len(std_coverage)), (mean_coverage - (.5 * std_coverage)), (mean_coverage + (.5 * std_coverage)), color='blue', alpha=0.2)
    plt.fill_between(range(len(std_coverage)), (mean_coverage - std_coverage), (mean_coverage + std_coverage), color='blue', alpha=0.1)
    plt.legend(loc='lower right')
    plt.title(title)
    plt.ylabel('Coverage')
    plt.xlabel('Generations')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.savefig(f'results/{dir_name}/{filename}-{exp_name}.png', transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
def plot_compare(arrs: List[np.ndarray],
                 colors: List[str],
                 linestyles: List[str],
                 labels: List[str],
                 title: str,
                 ylabel: str,
                 partial_filename: str,
                 dir_name: str,
                 add_area: bool = True,
                 use_ci: bool = False,
                 use_se: bool = False,
                 plot_elites: bool = True,
                 legend_loc: str = 'lower right'):
    w, h = matplotlib.figure.figaspect(9/16)
    fig = plt.figure(figsize=(w,h))
    
    plt.grid()
    for arr, color, label, linestyle in zip(arrs, colors, labels, linestyles):
        if len(arr.shape) == 3:
            values = arr[:,:,0] if plot_elites else arr[:,:,1]
        elif len(arr.shape) == 2:
            values = arr[:]
        else:
            raise NotImplementedError(f'Unexpected array shape: {arr.shape}')
        values_mean = np.nanmean(values, axis=0)
        
        plt.plot(range(len(values_mean)), values_mean, label=label, c=color, lw=2, linestyle=linestyle)
        if add_area:
            if use_ci:
                confidence = 0.95
                res = bootstrap((values, ), np.nanmean, confidence_level=confidence, method='BCa')
                lower_confidence_intervals, upper_confidence_intervals = res.confidence_interval.low, res.confidence_interval.high
                plt.fill_between(range(len(values_mean)), lower_confidence_intervals, upper_confidence_intervals, color=color, alpha=0.1)
            elif use_se:
                values_se = sem(values, axis=0)
                plt.fill_between(range(len(values_se)), (values_mean - values_se), (values_mean + values_se), color=color, alpha=0.1)
            else:
                values_std = np.std(values, axis=0)
                plt.fill_between(range(len(values_std)), (values_mean - values_std), (values_mean + values_std), color=color, alpha=0.1)
    
    plt.legend(loc=legend_loc)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel('Generations')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.savefig(f'results/{dir_name}/comparison-{partial_filename}.png', transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
def print_stats(arr: np.ndarray,
                title: str,
                elites: bool = True,
                overall: bool = False):
    if len(arr.shape) == 3:
        values = arr[:,:,0] if elites else arr[:,:,1]
    elif len(arr.shape) == 2:
        values = arr[:]
    else:
        raise NotImplementedError(f'Unexpected array shape: {arr.shape}')
    if np.isnan(arr).all():
        print(f'{title}: NaN')
    else:
        values_mean = np.nanmean(values, axis=0)
        values_std = np.nanstd(values, axis=0)
        v1 = values_mean[-1] if not overall else np.mean(values_mean)
        v2 = values_std[-1] if not overall else np.mean(values_std)
        print(f'{title}: {np.round(v1, 4)} +- {np.round(v2, 4)}')

In [ ]:
def save_metrics(f_fitnesses_hist,
                 i_fitnesses_hist,
                 estimator_errs,
                 percentage_new_feas,
                 exp_name,
                 dir_name,
                 qd_scores = None):
    ffs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(f_fitnesses_hist):
        for g, gv in enumerate(rv):
            ffs[r, g, :] = gv
    ifs = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
    for r, rv in enumerate(i_fitnesses_hist):
        for g, gv in enumerate(rv):
            ifs[r, g, :] = gv
    with open(f'results/{dir_name}/{exp_name}_metrics.npz', 'wb') as f:
        np.savez(f, ffs, ifs)
    ers = np.empty(shape=(N_RUNS, N_GENS))
    for r, rv in enumerate(estimator_errs):
            ers[r, :] = rv
    with open(f'results/{dir_name}/{exp_name}_estimator_errs.npz', 'wb') as f:
        np.savez(f, ers)
    if isinstance(percentage_new_feas[0], list):
        pfs = np.empty(shape=(N_RUNS, N_GENS, 2))
    else:
        pfs = np.empty(shape=(N_RUNS, N_GENS))
    for r, rv in enumerate(percentage_new_feas):
        if isinstance(rv, list):
            for g, gv in enumerate(rv):
                pfs[r, g, :] = gv
        else:
            pfs[r, :] = rv
    with open(f'results/{dir_name}/{exp_name}_percentage_new_feas.npz', 'wb') as f:
        np.savez(f, pfs)
    if qd_scores is not None:
        qds = np.empty(shape=(N_RUNS, 1 + N_GENS, 2))
        for r, rv in enumerate(qd_scores):
            for g, gv in enumerate(rv):
                qds[r, g, :] = gv
        with open(f'results/{dir_name}/{exp_name}_qds.npz', 'wb') as f:
            np.savez(f, qds)
        
def save_coverage(f_coverages,
                  i_coverages,
                  exp_name,
                  dir_name):
    fcs = np.empty(shape=(N_RUNS, 1 + N_GENS))
    for r, rv in enumerate(f_coverages):
        fcs[r, :] = rv
    ics = np.empty(shape=(N_RUNS, 1 + N_GENS))
    for r, rv in enumerate(i_coverages):
        ics[r, :] = rv
    with open(f'results/{dir_name}/{exp_name}_coverages.npz', 'wb') as f:
        np.savez(f, fcs, ics)

In [ ]:
def save_solution_stats(cs: CandidateSolution,
                        exp_name: str,
                        dir_name: str) -> None:
    with open(f'results/{dir_name}/{exp_name}_atoms.log', 'a') as f:
        f.write(f'\n\n{cs.string}')
        f.write(f'\nFitness: {cs.fitness}')

## Experiments settings

In [ ]:
run_experiments = {
    1: False,
    2: False,
    3: False,
    4: False,
    5: False,
    6: False,
    7: False
}
plot_experiments_results = {
    1: False,
    2: False,
    3: False,
    4: False,
    5: False,
    6: False,
    7: False
}

In [ ]:
expander.initialize(rules=lsystem.hl_solver.parser.rules)

In [ ]:
feasible_fitnesses = [
    Fitness(name='BoxFilling',
            f=box_filling_fitness,
            bounds=(0, 1)),
    Fitness(name='FuncionalBlocks',
            f=func_blocks_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMediumProportions',
            f=mame_fitness,
            bounds=(0, 1)),
    Fitness(name='MajorMinimumProportions',
            f=mami_fitness,
            bounds=(0, 1))
]

## Experiment 1

Run standard FI-2Pop against variant FI-2Pop

### FI-2Pop variant

In [ ]:
class FI2PopVariantSolver:
    def __init__(self,
                 feasible_fitnesses: List[Fitness],
                 lsystem: LSystem,
                 buffer: Buffer,
                 estimator: Optional[Union[GaussianEstimator, MLPEstimator]] = None):
        """Create the FI2Pop solver.

        Args:
            feasible_fitnesses (List[Fitness]): The list of fitnesses.
            lsystem (LSystem): The L-system object.
        """
        self.feasible_fitnesses = feasible_fitnesses
        self.lsystem = lsystem
        self.ftop = []
        self.itop = []
        self.fmean = []
        self.imean = []

        self.ffs, self.ifs = [], []

        # number of total soft constraints
        self.nsc = [c for c in self.lsystem.all_hl_constraints if c.level == ConstraintLevel.SOFT_CONSTRAINT]
        self.nsc = [c for c in self.lsystem.all_ll_constraints if c.level == ConstraintLevel.SOFT_CONSTRAINT]
        self.nsc = len(self.nsc) * 0.5
        
        self.perc_feas_infeas = []
        
        self.estimator = estimator
        self.buffer = buffer
        self.max_f_fitness = sum([f.bounds[1] for f in self.feasible_fitnesses])

    def reset(self):
        self.ftop = []
        self.itop = []
        self.fmean = []
        self.imean = []
        self.ffs, self.ifs = [], []
        
        self.perc_feas_infeas = []
        
        self.estimator = MLPEstimator(len(self.feasible_fitnesses), 1)
        self.buffer = Buffer(merge_method=self.buffer._merge)

    def _compute_fitness(self,
                         cs: CandidateSolution,
                         extra_args: Dict[str, Any]) -> List[float]:
        """Compute the fitness of a single candidate solution.

        Args:
            cs (CandidateSolution): The candidate solution.
            extra_args (Dict[str, Any]): Additional arguments used in the fitness function.

        Returns:
            float: The fitness value.
        """
        if cs.fitness == []:
            if cs.is_feasible:
                try:
                    cs.fitness = [f(cs, extra_args) for f in self.feasible_fitnesses]
                except Exception as e:
                    print(f"Ignored fitness due to {str(e)}")
                    cs.fitness = [0 for _ in self.feasible_fitnesses]
                    
                cs.representation = cs.fitness[:]
                x, y, z = cs.content._max_dims
                cs.representation.extend([x / MAX_X_SIZE,
                                          y / MAX_Y_SIZE,
                                          z / MAX_Z_SIZE])
            else:
                cs.representation = [f(cs, extra_args) for f in [Fitness(name='BoxFilling', f=box_filling_fitness, bounds=(0, 1)),
                                                                 Fitness(name='FuncionalBlocks', f=func_blocks_fitness, bounds=(0, 1)),
                                                                 Fitness(name='MajorMediumProportions', f=mame_fitness, bounds=(0, 1)),
                                                                 Fitness(name='MajorMinimumProportions', f=mami_fitness, bounds=(0, 1))]]
        if cs.is_feasible:
            return sum([self.feasible_fitnesses[i].weight * cs.fitness[i] for i in range(len(cs.fitness))])
        else:
            if self.estimator is not None:
                if isinstance(self.estimator, GaussianEstimator):
                    return self.estimator.predict(x=np.asarray(cs.fitness))
                elif isinstance(self.estimator, MLPEstimator):
                    if self.estimator.is_trained:
                        with th.no_grad():
                            return self.estimator(th.tensor(cs.representation).float()).numpy()[0]
                    else:
                        return EPSILON_F
                elif isinstance(self.estimator, QuantileEstimator):
                    if self.estimator.is_trained:
                        with th.no_grad():
                            # set fitness to (3,) array (min, median, max)
                            cs.fitness = self.estimator(th.tensor(cs.representation).float().unsqueeze(0)).numpy()[0]
                            return cs.fitness[self.infeas_fitness_idx]  # set c_itness to median by default
                    else:
                        cs.fitness = [EPSILON_F, EPSILON_F, EPSILON_F]
                        return cs.fitness[self.infeas_fitness_idx]
                else:
                    raise NotImplementedError(f'Unrecognized estimator type: {type(self.estimator)}.')
            else:
                return cs.ncv

    def _generate_initial_populations(self,
                                      pops_size: int = POP_SIZE,
                                      n_retries: int = N_RETRIES) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Generate the initial populations.

        Args:
            pops_size (int, optional): The size of the population. Defaults to POP_SIZE.
            n_retries (int, optional): The number of retries. Defaults to N_RETRIES.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and Infeasible populations.
        """
        feasible_pop, infeasible_pop = [], []
        self.lsystem.disable_sat_check()
        with trange(n_retries, desc='Initialization ') as iterations:
            for i in iterations:
                solutions = self.lsystem.apply_rules(starting_strings=['head', 'body', 'tail'],
                                                     iterations=[1, N_ITERATIONS, 1],
                                                     create_structures=False,
                                                     make_graph=False)
                subdivide_solutions(lcs=solutions,
                                    lsystem=self.lsystem)
                for cs in solutions:
                    if cs.is_feasible and len(feasible_pop) < pops_size and cs not in feasible_pop:
                        feasible_pop.append(cs)
                        cs.c_fitness = self._compute_fitness(cs=cs,
                                                           extra_args={
                                                               'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                               }) + (self.nsc - cs.ncv)
                    elif not cs.is_feasible and len(infeasible_pop) < pops_size and cs not in feasible_pop:
                        cs.c_fitness = self._compute_fitness(cs=cs,
                                                             extra_args={
                                                                 'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                 })
                        infeasible_pop.append(cs)
                iterations.set_postfix(ordered_dict={'fpop-size': f'{len(feasible_pop)}/{pops_size}',
                                                     'ipop-size': f'{len(infeasible_pop)}/{pops_size}'},
                                       refresh=True)
                if i == n_retries or (len(feasible_pop) == pops_size and len(infeasible_pop) == pops_size):
                    break
        return feasible_pop, infeasible_pop

    def initialize(self,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Initialize the solver by generating the initial populations.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and Infeasible populations.
        """
        f_pop, i_pop = self._generate_initial_populations(pops_size=pops_size,
                                                          n_retries=n_retries)
        f_fitnesses = [cs.c_fitness for cs in f_pop]
        i_fitnesses = [cs.c_fitness for cs in i_pop]
        self.ftop.append(max(f_fitnesses))
        self.fmean.append(sum(f_fitnesses) / len(f_fitnesses))
        self.itop.append(max(i_fitnesses))
        self.imean.append(sum(i_fitnesses) / len(i_fitnesses))
        self.ffs.append([self.ftop[-1], self.fmean[-1]])
        self.ifs.append([self.itop[-1], self.imean[-1]])
        print(f'Created Feasible population of size {len(f_pop)}: t:{self.ftop[-1]};m:{self.fmean[-1]}')
        print(f'Created Infeasible population of size {len(i_pop)}: t:{self.itop[-1]};m:{self.imean[-1]}')
        return f_pop, i_pop

    def fi2pop(self,
               f_pop: List[CandidateSolution],
               i_pop: List[CandidateSolution],
               n_iter: int = N_GENS) -> Tuple[List[CandidateSolution], List[CandidateSolution]]:
        """Apply the FI2Pop algorithm to the given populations for `n_iter` steps.

        Args:
            f_pop (List[CandidateSolution]): The Feasible population.
            i_pop (List[CandidateSolution]): The Infeasible population.
            n_iter (int, optional): The number of iterations to run for. Defaults to N_GENS.

        Returns:
            Tuple[List[CandidateSolution], List[CandidateSolution]]: The Feasible and the Infeasible populations.
        """
        f_pool = []
        i_pool = []        
        with trange(n_iter, desc='Generation ') as gens:
            for gen in gens:
                # place the infeasible population in the infeasible pool
                i_pool.extend(i_pop)
                
                f_pool.extend(f_pop)
                
                # create offsprings from feasible population
                new_pool = create_new_pool(population=f_pop,
                                           generation=gen)
                # if feasible, add to feasible pool
                # if infeasible, add to infeasible pool
                subdivide_solutions(lcs=new_pool,
                                    lsystem=self.lsystem)
                for cs in new_pool:
                    
                    cs.age = CS_MAX_AGE
                    
                    if cs.is_feasible:
                        try:
                            cs.ll_string = self.lsystem.hl_to_ll(cs=cs).string
                            cs.c_fitness = self._compute_fitness(cs=cs,
                                                                extra_args={
                                                                    'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                }) + (self.nsc - cs.ncv)
                            f_pool.append(cs)
                        except Exception:
                            print(f'Feasible cs too big, is in f_pool: {cs in f_pool}')
                    else:
                        try:
                            cs.c_fitness = self._compute_fitness(cs=cs,
                                                                extra_args={
                                                                    'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                })
                            i_pool.append(cs)
                        except Exception:
                            print(f'Ineasible cs too big, is in f_pool: {cs in f_pool}')
                # place the infeasible population in the infeasible pool
                # i_pool.extend(i_pop)s                                  
                
                i_pool = list(set(i_pool))
                
                # reduce the infeasible pool if > pops_size
                if len(i_pool) > POP_SIZE:
                    i_pool = reduce_population(population=i_pool,
                                               to=POP_SIZE,
                                               minimize=False)
                # set the infeasible pool as the infeasible population
                i_pop[:] = i_pool[:]
                # create offsprings from infeasible population
                new_pool = create_new_pool(population=i_pop,
                                           generation=gen,
                                           minimize=False)
                # if feasible, add to feasible pool
                # if infeasible, add to infeasible pool
                subdivide_solutions(lcs=new_pool,
                                    lsystem=self.lsystem)
                for cs in new_pool:
                    
                    cs.age = CS_MAX_AGE
                    
                    if cs.is_feasible:
                        try:
                            cs.ll_string = self.lsystem.hl_to_ll(cs=cs).string
                            cs.c_fitness = self._compute_fitness(cs=cs,
                                                                extra_args={
                                                                    'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                }) + (self.nsc - cs.ncv)
                            f_pool.append(cs)
                        except Exception:
                            print(f'Feasible cs too big, is in f_pool: {cs in f_pool}')
                    else:
                        try:
                            cs.c_fitness = self._compute_fitness(cs=cs,
                                                                extra_args={
                                                                    'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                })
                            i_pool.append(cs)
                        except Exception:
                            print(f'Ineasible cs too big, is in i_pool: {cs in i_pool}')
                
                f_pool = list(set(f_pool)) 
                
                # Prepare dataset for estimator
                xs, ys = prepare_dataset(f_pop=f_pool)
                for x, y in zip(xs, ys):
                    self.buffer.insert(x=x,
                                       y=y / self.max_f_fitness)
                # If possible, train estimator
                try:
                    xs, ys = self.buffer.get()
                    if isinstance(self.estimator, GaussianEstimator):
                        self.estimator.fit(x=xs,
                                            y=ys)
                    elif isinstance(self.estimator, MLPEstimator):
                        train_estimator(self.estimator,
                                        xs=xs,
                                        ys=ys)
                    else:
                        raise NotImplementedError(f'Unrecognized estimator type {type(self.estimator)}.')
                except EmptyBufferException as e:
                    pass
                if self.estimator.is_trained:
                    # Reassign previous infeasible fitnesses
                        for cs in i_pool:
                            try:
                                cs.c_fitness = self._compute_fitness(cs=cs,
                                                                    extra_args={
                                                                        'alphabet': self.lsystem.ll_solver.atoms_alphabet
                                                                        })
                            except Exception:
                                print(f'Error in fitness reassignment; should not happen if all cs too big were not added correctly.')     
                
                # reduce the feasible pool if > pops_size
                if len(f_pool) > POP_SIZE:
                    f_pool = reduce_population(population=f_pool,
                                               to=POP_SIZE)
                # set the feasible pool as the feasible population
                f_pop[:] = f_pool[:]
                
                n_new_feas_infeas = 0
                for cs in f_pop:
                    if cs.age == CS_MAX_AGE:
                        if not cs.parents[0].is_feasible:
                            n_new_feas_infeas += 1
                    cs.age -= 1
                n_new_feas_infeas /= len(f_pop)
                
                # update tracking
                f_fitnesses = [cs.c_fitness for cs in f_pop]
                i_fitnesses = [cs.c_fitness for cs in i_pop]
                self.ftop.append(max(f_fitnesses))
                self.fmean.append(sum(f_fitnesses) / len(f_fitnesses))
                self.itop.append(max(i_fitnesses))
                self.imean.append(sum(i_fitnesses) / len(i_fitnesses))
                self.ffs.append([self.ftop[-1], self.fmean[-1]])
                self.ifs.append([self.itop[-1], self.imean[-1]])
                
                self.perc_feas_infeas.append(n_new_feas_infeas)
                
                gens.set_postfix(ordered_dict={'top-f': self.ftop[-1],
                                               'mean-f': self.fmean[-1],
                                               'top-i': self.itop[-1],
                                               'mean-i': self.imean[-1]},
                                 refresh=True)

        return f_pop, i_pop


### Run

In [ ]:
def run_experiment(solver,
                   exp_name,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    i_fitnesses_hist = []
    estimator_errors = []
    percentage_new_feas = []
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            solver.reset()         
            f_pop, i_pop = solver.initialize(pops_size=pops_size,
                                             n_retries=n_retries)
            f_pop, i_pop = solver.fi2pop(f_pop=f_pop,
                                         i_pop=i_pop,
                                         n_iter=N_GENS)
            f_fitnesses_hist.append(solver.ffs)
            i_fitnesses_hist.append(solver.ifs)
            percentage_new_feas.append(solver.perc_feas_infeas)
            f_fitnesses = [cs.c_fitness for cs in f_pop]
            i_fitnesses = [cs.ncv for cs in f_pop]
            if hasattr(solver, 'estimator') and solver.estimator.train_losses is not []:
                missing = N_GENS - len(solver.estimator.train_losses)
                to_add = [np.nan for _ in range(missing)]
                to_add.extend(solver.estimator.train_losses)
                estimator_errors.append(to_add)
            else:
                estimator_errors.append(np.nan)
            save_solution_stats(cs=f_pop[f_fitnesses.index(max(f_fitnesses))],
                                exp_name=f'{exp_name}-elites',
                                dir_name='cog_experiment01')
            for cs in f_pop:
                save_solution_stats(cs=cs,
                                    exp_name=exp_name,
                                    dir_name='cog_experiment01')
            iterations.set_postfix(ordered_dict={'f_fit': np.max(f_fitnesses),
                                                 'i_fit': np.max(i_fitnesses) if isinstance(solver, FI2PopVariantSolver) else np.min(i_fitnesses),
                                                 '%n': np.mean(percentage_new_feas[-1])},
                                   refresh=True)
            
    return f_fitnesses_hist, i_fitnesses_hist, estimator_errors, percentage_new_feas


In [ ]:
from pcgsepy.mapelites.buffer import mean_merge, max_merge, min_merge
from pcgsepy.fi2pop.fi2pop import FI2PopSolver

if run_experiments[1]:
    solvers, exp_names = [], []
    
    
    # solvers.append(FI2PopSolver(feasible_fitnesses=feasible_fitnesses,
    #                             lsystem=lsystem))
    # exp_names.append('standard-fi2pop')

    # for merge_method in [mean_merge, max_merge, min_merge]:
    for merge_method in [min_merge, max_merge]:
        buffer = Buffer(merge_method=merge_method)
        solver = FI2PopVariantSolver(feasible_fitnesses=feasible_fitnesses,
                                    lsystem=lsystem,
                                    buffer=buffer,
                                    estimator=MLPEstimator(len(feasible_fitnesses), 1))
        exp_name = f'variant-fi2pop-{buffer._merge.__name__}'
        
        solvers.append(solver)
        exp_names.append(exp_name)

    for solver, exp_name in zip(solvers, exp_names):
        f_fitnesses_hist, i_fitnesses_hist, estimator_errors, percentage_new_feas = run_experiment(solver=solver,
                                                                                                   exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    estimator_errs=estimator_errors,
                    percentage_new_feas=percentage_new_feas,
                    exp_name=exp_name,
                    dir_name='cog_experiment01')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                   exp_name=exp_name,
                                   dir_name='cog_experiment01')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                     exp_name=exp_name,
                                     dir_name='cog_experiment01')


In [ ]:
if plot_experiments_results[1]:
    experiments = {'standard-fi2pop': {}, 
                'variant-fi2pop-max_merge': {},
                'variant-fi2pop-mean_merge': {},
                'variant-fi2pop-min_merge': {}
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment01/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment01/{experiment}_estimator_errs.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['errs'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment01/{experiment}_percentage_new_feas.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['perc'] = npzfile['arr_0']

    feas_arrs = []
    infeas_arrs = []
    errs = []
    perc = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        errs.append(experiments[experiment]['errs'])
        perc.append(np.cumsum(experiments[experiment]['perc'], axis=1))
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['errs'],
                    title=f'{experiment}-estimator-errs',
                    elites=False)
        print_stats(arr=experiments[experiment]['perc'],
                    title=f'{experiment}-percentage-feas-increase',
                    elites=False,
                    overall=True)
        
    colors = ['darkgreen', 'blue', 'purple', 'red']
    linestyles = ['solid', 'dashed', 'dotted', 'dashdot']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=errs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Average estimator losses comparison ({N_RUNS} runs)',
                ylabel='Loss',
                partial_filename='avg-estimator-errs',
                plot_elites=True,
                add_area=False,
                legend_loc='upper right')
    
    plot_compare(arrs=perc,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment01',
                title=f'Average percentage feasible solution w/ infeasible parents ({N_RUNS} runs)',
                ylabel='Amount (%)',
                partial_filename='avg-percentage-feas-increase',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

## Experiment 2

Run standard MAP-Elites against variant MAP-Elites (w/ RandomEmitter).

In [ ]:
def run_experiment(mapelites: MAPElites,
                   exp_name: str,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    f_coverages_hist = []
    i_fitnesses_hist = []
    i_coverages_hist = []
    estimator_errors_hist = []
    percentage_new_feas_hist = []
    qd_scores_hist = []
    with open('estimators_perf.log', 'a') as f:
        f.write(f'Running experiment {exp_name}\n')
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            with open('estimators_perf.log', 'a') as f:
                f.write(f'Run {n+1}/`{N_RUNS}\n')
            f_fitnesses, i_fitnesses = [], []
            f_coverages, i_coverages = [], []
            qd_scores, percentage_new_feas = [], []
            mapelites.reset(lcs=[])
            mapelites.hull_builder = None
            mapelites.generate_initial_populations(pops_size=pops_size,
                                                   n_retries=n_retries)
            f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
            i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
            qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
            fc_curr, total = mapelites.get_coverage(pop='feasible')
            ic_curr, _ = mapelites.get_coverage(pop='infeasible')
            f_coverages.append(fc_curr)
            i_coverages.append(ic_curr)
            with trange(N_GENS, desc=f'Running MAP-Elites (run {n+1})...') as gens:
                for gen in gens:
                    mapelites.emitter_step(gen=gen)
                    f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
                    i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
                    fc_curr, total = mapelites.get_coverage(pop='feasible')
                    ic_curr, _ = mapelites.get_coverage(pop='infeasible')
                    f_coverages.append(fc_curr)
                    i_coverages.append(ic_curr)                    
                    if gen == N_GENS - 1:
                        f_coverages = [x / total for x in f_coverages]
                        i_coverages = [x / total for x in i_coverages]
                    
                    percentage_new_feas.append(mapelites.get_new_feas_with_unfeas_parents())
                                        
                    qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
                    
                    gens.set_postfix(ordered_dict={'f_top_fit': f_fitnesses[-1][0],
                                                   'f_avg_fit': f_fitnesses[-1][1],
                                                   'i_top_fit': i_fitnesses[-1][0],
                                                   'i_avg_fit': i_fitnesses[-1][1],
                                                   '%n': percentage_new_feas[-1][0] / percentage_new_feas[-1][1],
                                                   'f_qd': qd_scores[-1][0],
                                                   'i_qd': qd_scores[-1][1]},
                                     refresh=True)
                f_fitnesses_hist.append(f_fitnesses)
                i_fitnesses_hist.append(i_fitnesses)
                f_coverages_hist.append(f_coverages)
                i_coverages_hist.append(i_coverages)
                
                percentage_new_feas_hist.append(percentage_new_feas)
                qd_scores_hist.append(qd_scores)
                
                if mapelites.estimator:
                    missing = N_GENS - len(mapelites.estimator.train_losses)
                    if missing >= 0:
                        to_add = [np.nan for _ in range(missing)]
                        to_add.extend(mapelites.estimator.train_losses)
                    else:
                        to_add = mapelites.estimator.train_losses[:50]
                else:
                    to_add = [np.nan for _ in range(N_GENS)]
                estimator_errors_hist.append(to_add)
                
                save_solution_stats(cs=mapelites.get_random_elite(pop='feasible'),
                                    exp_name=f'{exp_name}-elites',
                                    dir_name='cog_experiment02')
                
                for i in range(mapelites.bins.shape[0]):
                    for j in range(mapelites.bins.shape[1]):
                        for cs in mapelites.bins[i, j]._feasible:
                            save_solution_stats(cs=cs,
                                                exp_name=exp_name,
                                                dir_name='cog_experiment02')
                
            iterations.set_postfix(ordered_dict={'f_coverage': f_coverages_hist[-1][-1],
                                                 'i_coverages': i_coverages_hist[-1][-1]})
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment02/{exp_name}-fitness-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment02/{exp_name}-fitness-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment02/{exp_name}-coverage-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment02/{exp_name}-coverage-run{str(n).zfill(2)}')
    return f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist


In [ ]:
from pcgsepy.mapelites.buffer import mean_merge, max_merge, min_merge
from pcgsepy.mapelites.behaviors import BehaviorCharacterization, mame, mami
from pcgsepy.mapelites.emitters import RandomEmitter

if run_experiments[2]:
    l_mapelites, exp_names = [], []

    behavior_descriptors = (BehaviorCharacterization(name='mame',
                                                     func=mame,
                                                     bounds=(0, 20)),
                            BehaviorCharacterization(name='mami',
                                                     func=mami,
                                                     bounds=(0, 20)))
    
    for merge_method in [mean_merge, max_merge, min_merge]:
        buffer = Buffer(merge_method=merge_method)
        mapelites = MAPElites(lsystem=lsystem,
                              feasible_fitnesses=feasible_fitnesses,
                              estimator=MLPEstimator(xshape=len(feasible_fitnesses),
                                                     yshape=1),
                              buffer=buffer,
                              behavior_descriptors=behavior_descriptors,
                              n_bins=(32,32),
                              emitter=RandomEmitter())
        mapelites.allow_aging = False
        mapelites.allow_res_increase = False
        exp_name = f'variant-mapelites-{buffer._merge.__name__}'
        l_mapelites.append(mapelites)
        exp_names.append(exp_name)

    l_mapelites.append(MAPElites(lsystem=lsystem,
                              feasible_fitnesses=feasible_fitnesses,
                              estimator=None,
                              buffer=Buffer(merge_method=mean_merge),  # unused anyways
                              behavior_descriptors=behavior_descriptors,
                              n_bins=(32,32),
                              emitter=RandomEmitter()))
    l_mapelites[-1].allow_aging = False
    l_mapelites[-1].allow_res_increase = False
    exp_names.append('standard-mapelites')

    for mapelites, exp_name in zip(l_mapelites, exp_names):
        f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist = run_experiment(mapelites=mapelites,
                                                                                                                                                                 exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    estimator_errs=estimator_errors_hist,
                    percentage_new_feas=percentage_new_feas_hist,
                    qd_scores=qd_scores_hist,                    
                    exp_name=exp_name,
                    dir_name='cog_experiment02')
        save_coverage(f_coverages=f_coverages_hist,
                      i_coverages=i_coverages_hist,
                      exp_name=exp_name,
                      dir_name='cog_experiment02')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                   exp_name=exp_name,
                                   dir_name='cog_experiment02')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                     exp_name=exp_name,
                                     dir_name='cog_experiment02')
        plot_coverage(coverages_hist=f_coverages_hist,
                      title=f'Avg. Feasible coverage',
                      exp_name=exp_name,
                      filename='avg-feas-coverage',
                      dir_name='cog_experiment02')
        plot_coverage(coverages_hist=i_coverages_hist,
                      title=f'Avg. Infeasible coverage',
                      exp_name=exp_name,
                      filename='avg-infeas-coverage',
                      dir_name='cog_experiment02')


In [ ]:
if plot_experiments_results[2]:
    experiments = {'standard-mapelites': {}, 
                'variant-mapelites-max_merge': {},
                'variant-mapelites-mean_merge': {},
                'variant-mapelites-min_merge': {}
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment02/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']
        with open(f'results/cog_experiment02/{experiment}_coverages.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
            experiments[experiment]['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
    for experiment in experiments.keys():
        with open(f'results/cog_experiment02/{experiment}_estimator_errs.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['errs'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment02/{experiment}_percentage_new_feas.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['perc'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment02/{experiment}_qds.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['qds'] = npzfile['arr_0']
            
    feas_arrs = []
    infeas_arrs = []
    feas_coverages = []
    infeas_coverages = []
    errs = []
    perc = []
    qds = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        feas_coverages.append(experiments[experiment]['feasible_coverage'])
        infeas_coverages.append(experiments[experiment]['infeasible_coverage'])
        errs.append(experiments[experiment]['errs'])
        
        tmp = np.cumsum(experiments[experiment]['perc'], axis=1)
        tmp[:,:,0] = tmp[:,:,0] / tmp[:, :, 1]
               
        # perc.append(np.cumsum(experiments[experiment]['perc'], axis=1))
        perc.append(tmp[:,:,0])
        qds.append(experiments[experiment]['qds'])
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['feasible_coverage'],
                    title=f'{experiment}-feas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible_coverage'],
                    title=f'{experiment}-infeas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['errs'],
                    title=f'{experiment}-estimator-errs',
                    elites=False)
        print_stats(arr=tmp,
                    title=f'{experiment}-percentage-feas-increase',
                    elites=True)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=True,
                    overall=False)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-infeas-QD-score',
                    elites=False,
                    overall=False)
        
    colors = ['darkgreen', 'blue', 'purple', 'red']
    linestyles = ['solid', 'dashed', 'dotted', 'dashdot']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=feas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average Feasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-feas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=infeas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average Infeasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-infeas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=errs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average estimator losses comparison ({N_RUNS} runs)',
                ylabel='Loss',
                partial_filename='avg-estimator-errs',
                plot_elites=True,
                add_area=False,
                legend_loc='upper right')
    
    plot_compare(arrs=perc,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'Average percentage feasible solution w/ infeasible parents ({N_RUNS} runs)',
                ylabel='Amount (%)',
                partial_filename='avg-percentage-feas-increase',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'QD-score (feasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-feas',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment02',
                title=f'QD-score (infeasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-infeas',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

## Experiment 3

Run variant MAP-Elites w/ OptimisationEmitter.

In [ ]:
def run_experiment(mapelites: MAPElites,
                   exp_name: str,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    f_coverages_hist = []
    i_fitnesses_hist = []
    i_coverages_hist = []
    estimator_errors_hist = []
    percentage_new_feas_hist = []
    qd_scores_hist = []
    with open('estimators_perf.log', 'a') as f:
        f.write(f'Running experiment {exp_name}\n')
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            with open('estimators_perf.log', 'a') as f:
                f.write(f'Run {n+1}/`{N_RUNS}\n')
            f_fitnesses, i_fitnesses = [], []
            f_coverages, i_coverages = [], []
            qd_scores, percentage_new_feas = [], []
            mapelites.reset(lcs=[])
            mapelites.hull_builder = None
            mapelites.generate_initial_populations(pops_size=pops_size,
                                                   n_retries=n_retries)
            f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
            i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
            qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
            fc_curr, total = mapelites.get_coverage(pop='feasible')
            ic_curr, _ = mapelites.get_coverage(pop='infeasible')
            f_coverages.append(fc_curr)
            i_coverages.append(ic_curr)
            with trange(N_GENS, desc=f'Running MAP-Elites (run {n+1})...') as gens:
                for gen in gens:
                    mapelites.emitter_step(gen=gen)
                    f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
                    i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
                    fc_curr, total = mapelites.get_coverage(pop='feasible')
                    ic_curr, _ = mapelites.get_coverage(pop='infeasible')
                    f_coverages.append(fc_curr)
                    i_coverages.append(ic_curr)                    
                    if gen == N_GENS - 1:
                        f_coverages = [x / total for x in f_coverages]
                        i_coverages = [x / total for x in i_coverages]
                    
                    percentage_new_feas.append(mapelites.get_new_feas_with_unfeas_parents())
                                        
                    qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
                    
                    gens.set_postfix(ordered_dict={'f_top_fit': f_fitnesses[-1][0],
                                                   'f_avg_fit': f_fitnesses[-1][1],
                                                   'i_top_fit': i_fitnesses[-1][0],
                                                   'i_avg_fit': i_fitnesses[-1][1],
                                                   '%n': percentage_new_feas[-1][0] / percentage_new_feas[-1][1],
                                                   'f_qd': qd_scores[-1][0],
                                                   'i_qd': qd_scores[-1][1]},
                                     refresh=True)
                f_fitnesses_hist.append(f_fitnesses)
                i_fitnesses_hist.append(i_fitnesses)
                f_coverages_hist.append(f_coverages)
                i_coverages_hist.append(i_coverages)
                
                percentage_new_feas_hist.append(percentage_new_feas)
                qd_scores_hist.append(qd_scores)
                
                if mapelites.estimator:
                    missing = N_GENS - len(mapelites.estimator.train_losses)
                    if missing >= 0:
                        to_add = [np.nan for _ in range(missing)]
                        to_add.extend(mapelites.estimator.train_losses)
                    else:
                        to_add = mapelites.estimator.train_losses[:50]
                else:
                    to_add = [np.nan for _ in range(N_GENS)]
                estimator_errors_hist.append(to_add)
                
                save_solution_stats(cs=mapelites.get_random_elite(pop='feasible'),
                                    exp_name=f'{exp_name}-elites',
                                    dir_name='cog_experiment03')
                
                for i in range(mapelites.bins.shape[0]):
                    for j in range(mapelites.bins.shape[1]):
                        for cs in mapelites.bins[i, j]._feasible:
                            save_solution_stats(cs=cs,
                                                exp_name=exp_name,
                                                dir_name='cog_experiment03')
                
            iterations.set_postfix(ordered_dict={'f_coverage': f_coverages_hist[-1][-1],
                                                 'i_coverages': i_coverages_hist[-1][-1]})
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment03/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment03/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment03/{exp_name}-coverage-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment03/{exp_name}-coverage-run{str(n).zfill(2)}')
    return f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist


In [ ]:
if run_experiments[3]:
    l_mapelites, exp_names = [], []

    behavior_descriptors = (BehaviorCharacterization(name='mame',
                                                     func=mame,
                                                     bounds=(0, 20)),
                            BehaviorCharacterization(name='mami',
                                                     func=mami,
                                                     bounds=(0, 20)))
    
    for merge_method in [mean_merge, max_merge, min_merge]:
        buffer = Buffer(merge_method=merge_method)
        mapelites = MAPElites(lsystem=lsystem,
                              feasible_fitnesses=feasible_fitnesses,
                              estimator=MLPEstimator(xshape=len(feasible_fitnesses),
                                                     yshape=1),
                              buffer=buffer,
                              behavior_descriptors=behavior_descriptors,
                              n_bins=(32,32),
                              emitter=OptimisingEmitter())
        mapelites.allow_aging = False
        mapelites.allow_res_increase = False
        exp_name = f'variant-optim-mapelites-{buffer._merge.__name__}'
        l_mapelites.append(mapelites)
        exp_names.append(exp_name)

    for mapelites, exp_name in zip(l_mapelites, exp_names):
        f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist = run_experiment(mapelites=mapelites,
                                                                                                                                                                 exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    estimator_errs=estimator_errors_hist,
                    percentage_new_feas=percentage_new_feas_hist,
                    qd_scores=qd_scores_hist,  
                    exp_name=exp_name,
                    dir_name='cog_experiment03')
        save_coverage(f_coverages=f_coverages_hist,
                      i_coverages=i_coverages_hist,      
                      exp_name=exp_name,
                      dir_name='cog_experiment03')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                   exp_name=exp_name,
                                   dir_name='cog_experiment03')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                     exp_name=exp_name,
                                     dir_name='cog_experiment03')
        plot_coverage(coverages_hist=f_coverages_hist,
                      title=f'Avg. Feasible coverage',
                      exp_name=exp_name,
                      filename='avg-feas-coverage',
                      dir_name='cog_experiment03')
        plot_coverage(coverages_hist=i_coverages_hist,
                      title=f'Avg. Infeasible coverage',
                      exp_name=exp_name,
                      filename='avg-infeas-coverage',
                      dir_name='cog_experiment03')


In [ ]:
if plot_experiments_results[3]:
    experiments = {
                'variant-optim-mapelites-max_merge': {},
                'variant-optim-mapelites-mean_merge': {},
                'variant-optim-mapelites-min_merge': {}
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment03/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']
        with open(f'results/cog_experiment03/{experiment}_coverages.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
            experiments[experiment]['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
    for experiment in experiments.keys():
        with open(f'results/cog_experiment03/{experiment}_estimator_errs.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['errs'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment03/{experiment}_percentage_new_feas.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['perc'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment03/{experiment}_qds.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['qds'] = npzfile['arr_0']

    feas_arrs = []
    infeas_arrs = []
    feas_coverages = []
    infeas_coverages = []
    errs = []
    perc = []
    qds = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        feas_coverages.append(experiments[experiment]['feasible_coverage'])
        infeas_coverages.append(experiments[experiment]['infeasible_coverage'])
        errs.append(experiments[experiment]['errs'])
        
        tmp = np.cumsum(experiments[experiment]['perc'], axis=1)
        tmp[:,:,0] = tmp[:,:,0] / tmp[:, :, 1]
               
        perc.append(tmp[:,:,0])
        qds.append(experiments[experiment]['qds'])
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['feasible_coverage'],
                    title=f'{experiment}-feas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible_coverage'],
                    title=f'{experiment}-infeas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['errs'],
                    title=f'{experiment}-estimator-errs',
                    elites=False)
        print_stats(arr=experiments[experiment]['perc'],
                    title=f'{experiment}-percentage-feas-increase',
                    elites=False,
                    overall=True)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=True,
                    overall=False)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=False,
                    overall=False)
        
    colors = ['darkgreen', 'blue', 'purple']
    linestyles = ['dashed', 'dotted', 'dashdot']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=feas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average Feasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-feas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=infeas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average Infeasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-infeas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=errs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average estimator losses comparison ({N_RUNS} runs)',
                ylabel='Loss',
                partial_filename='avg-estimator-errs',
                plot_elites=True,
                add_area=False,
                legend_loc='upper right')
    
    plot_compare(arrs=perc,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'Average percentage feasible solution w/ infeasible parents ({N_RUNS} runs)',
                ylabel='Amount (%)',
                partial_filename='avg-percentage-feas-increase',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'QD-score (feasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-feas',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment03',
                title=f'QD-score (infeasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-infeas',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

## Experiment 4

Run standard variants FI-2Pop with UCB or LCB.

### Run

In [ ]:
def run_experiment(solver,
                   exp_name,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    i_fitnesses_hist = []
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            solver.reset()         
            f_pop, i_pop = solver.initialize(pops_size=pops_size,
                                             n_retries=n_retries)
            f_pop, i_pop = solver.fi2pop(f_pop=f_pop,
                                         i_pop=i_pop,
                                         n_iter=N_GENS)
            f_fitnesses_hist.append(solver.ffs)
            i_fitnesses_hist.append(solver.ifs)
            f_fitnesses = [cs.c_fitness for cs in f_pop]
            i_fitnesses = [cs.ncv for cs in f_pop]
            save_solution_stats(cs=f_pop[f_fitnesses.index(max(f_fitnesses))],
                                exp_name=exp_name,
                                dir_name='cog_experiment04')
            iterations.set_postfix(ordered_dict={'f_fit': np.max(f_fitnesses),
                                                 'i_fit': np.max(i_fitnesses) if isinstance(solver, FI2PopVariantSolver) else np.min(i_fitnesses)},
                                   refresh=True)
            
    return f_fitnesses_hist, i_fitnesses_hist


In [ ]:
if run_experiments[4]:
    solvers, exp_names = [], []

    for bound in ['upper', 'lower']:
        for merge_method in [mean_merge]:
            buffer = Buffer(merge_method=merge_method)
            solver = FI2PopVariantSolver(feasible_fitnesses=feasible_fitnesses,
                                        lsystem=lsystem,
                                        buffer=buffer,
                                        estimator=GaussianEstimator(bound='upper',
                                                                    kernel=None,
                                                                    max_f=len(feasible_fitnesses) + 0.5,
                                                                    min_f=EPSILON_F))
            exp_name = f'variant-fi2pop-{buffer._merge.__name__}-' + 'ucb' if bound == 'upper' else 'lcb'
        
        solvers.append(solver)
        exp_names.append(exp_name)

    for solver, exp_name in zip(solvers, exp_names):
        f_fitnesses_hist, i_fitnesses_hist = run_experiment(solver=solver,
                                                            exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    exp_name=exp_name,
                    dir_name='cog_experiment04')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                exp_name=exp_name,
                                dir_name='cog_experiment04')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                    exp_name=exp_name,
                                    dir_name='cog_experiment04')


In [ ]:
if plot_experiments_results[4]:
    experiments = {
                'variant-fi2pop-mean_merge-ucb': {},
                'variant-fi2pop-mean_merge-lcb': {},
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment04/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']

    feas_arrs = []
    infeas_arrs = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        
    colors = ['red', 'red']
    linestyles = ['dashed', 'dotted']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment04',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment04',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment04',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment04',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

## Experiment 5

Run variant MAP-Elites with population-specific optimisers.

In [ ]:
def run_experiment(mapelites: MAPElites,
                   exp_name: str,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    f_coverages_hist = []
    i_fitnesses_hist = []
    i_coverages_hist = []
    estimator_errors_hist = []
    percentage_new_feas_hist = []
    qd_scores_hist = []
    with open('estimators_perf.log', 'a') as f:
        f.write(f'Running experiment {exp_name}\n')
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            with open('estimators_perf.log', 'a') as f:
                f.write(f'Run {n+1}/`{N_RUNS}\n')
            f_fitnesses, i_fitnesses = [], []
            f_coverages, i_coverages = [], []
            qd_scores, percentage_new_feas = [], []
            mapelites.reset(lcs=[])
            mapelites.hull_builder = None
            mapelites.generate_initial_populations(pops_size=pops_size,
                                                   n_retries=n_retries)
            f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
            i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
            qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
            fc_curr, total = mapelites.get_coverage(pop='feasible')
            ic_curr, _ = mapelites.get_coverage(pop='infeasible')
            f_coverages.append(fc_curr)
            i_coverages.append(ic_curr)
            with trange(N_GENS, desc=f'Running MAP-Elites (run {n+1})...') as gens:
                for gen in gens:
                    mapelites.emitter_step(gen=gen)
                    f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
                    i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
                    fc_curr, total = mapelites.get_coverage(pop='feasible')
                    ic_curr, _ = mapelites.get_coverage(pop='infeasible')
                    f_coverages.append(fc_curr)
                    i_coverages.append(ic_curr)                    
                    if gen == N_GENS - 1:
                        f_coverages = [x / total for x in f_coverages]
                        i_coverages = [x / total for x in i_coverages]
                    
                    percentage_new_feas.append(mapelites.get_new_feas_with_unfeas_parents())
                                        
                    qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
                    
                    
                    gens.set_postfix(ordered_dict={'f_top_fit': f_fitnesses[-1][0],
                                                   'f_avg_fit': f_fitnesses[-1][1],
                                                   'i_top_fit': i_fitnesses[-1][0],
                                                   'i_avg_fit': i_fitnesses[-1][1],
                                                   '%n': percentage_new_feas[-1][0] / percentage_new_feas[-1][1],
                                                   'f_qd': qd_scores[-1][0],
                                                   'i_qd': qd_scores[-1][1]},
                                     refresh=True)
                f_fitnesses_hist.append(f_fitnesses)
                i_fitnesses_hist.append(i_fitnesses)
                f_coverages_hist.append(f_coverages)
                i_coverages_hist.append(i_coverages)
                
                percentage_new_feas_hist.append(percentage_new_feas)
                qd_scores_hist.append(qd_scores)
                
                if mapelites.estimator:
                    missing = N_GENS - len(mapelites.estimator.train_losses)
                    if missing >= 0:
                        to_add = [np.nan for _ in range(missing)]
                        to_add.extend(mapelites.estimator.train_losses)
                    else:
                        to_add = mapelites.estimator.train_losses[:50]
                else:
                    to_add = [np.nan for _ in range(N_GENS)]
                estimator_errors_hist.append(to_add)
                
                save_solution_stats(cs=mapelites.get_random_elite(pop='feasible'),
                                    exp_name=f'{exp_name}-elites',
                                    dir_name='cog_experiment05')
                
                for i in range(mapelites.bins.shape[0]):
                    for j in range(mapelites.bins.shape[1]):
                        for cs in mapelites.bins[i, j]._feasible:
                            save_solution_stats(cs=cs,
                                                exp_name=exp_name,
                                                dir_name='cog_experiment05')
                
            
            iterations.set_postfix(ordered_dict={'f_coverage': f_coverages_hist[-1][-1],
                                                 'i_coverages': i_coverages_hist[-1][-1]})
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment05/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment05/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment05/{exp_name}-coverage-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment05/{exp_name}-coverage-run{str(n).zfill(2)}')
    return f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist


In [ ]:
if run_experiments[5]:
    l_mapelites, exp_names = [], []

    behavior_descriptors = (BehaviorCharacterization(name='mame',
                                                     func=mame,
                                                     bounds=(0, 20)),
                            BehaviorCharacterization(name='mami',
                                                     func=mami,
                                                     bounds=(0, 20)))
    
    for merge_method in [mean_merge, max_merge, min_merge]:
        buffer = Buffer(merge_method=merge_method)
        mapelites = MAPElites(lsystem=lsystem,
                              feasible_fitnesses=feasible_fitnesses,
                              estimator=MLPEstimator(xshape=len(feasible_fitnesses),
                                                     yshape=1),
                              buffer=buffer,
                              behavior_descriptors=behavior_descriptors,
                              n_bins=(32,32),
                              emitter=OptimisingEmitterV2())
        mapelites.allow_aging = False
        mapelites.allow_res_increase = False
        exp_name = f'variant-optim-mapelites-{buffer._merge.__name__}'
        l_mapelites.append(mapelites)
        exp_names.append(exp_name)

    for mapelites, exp_name in zip(l_mapelites, exp_names):
        f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist = run_experiment(mapelites=mapelites,
                                                                                                                                                                    exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    estimator_errs=estimator_errors_hist,
                    percentage_new_feas=percentage_new_feas_hist,
                    qd_scores=qd_scores_hist,       
                    exp_name=exp_name,
                    dir_name='cog_experiment05')
        save_coverage(f_coverages=f_coverages_hist,
                      i_coverages=i_coverages_hist,
                      exp_name=exp_name,
                      dir_name='cog_experiment05')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                   exp_name=exp_name,
                                   dir_name='cog_experiment05')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                     exp_name=exp_name,
                                     dir_name='cog_experiment05')
        plot_coverage(coverages_hist=f_coverages_hist,
                      title=f'Avg. Feasible coverage',
                      exp_name=exp_name,
                      filename='avg-feas-coverage',
                      dir_name='cog_experiment05')
        plot_coverage(coverages_hist=i_coverages_hist,
                      title=f'Avg. Infeasible coverage',
                      exp_name=exp_name,
                      filename='avg-infeas-coverage',
                      dir_name='cog_experiment05')


In [ ]:
if plot_experiments_results[5]:
    experiments = {
                'variant-optim-mapelites-max_merge': {},
                'variant-optim-mapelites-mean_merge': {},
                'variant-optim-mapelites-min_merge': {}
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment05/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']
        with open(f'results/cog_experiment05/{experiment}_coverages.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
            experiments[experiment]['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
    for experiment in experiments.keys():
        with open(f'results/cog_experiment05/{experiment}_estimator_errs.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['errs'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment05/{experiment}_percentage_new_feas.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['perc'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment05/{experiment}_qds.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['qds'] = npzfile['arr_0']
        
    feas_arrs = []
    infeas_arrs = []
    feas_coverages = []
    infeas_coverages = []
    errs = []
    perc = []
    qds = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        feas_coverages.append(experiments[experiment]['feasible_coverage'])
        infeas_coverages.append(experiments[experiment]['infeasible_coverage'])
        errs.append(experiments[experiment]['errs'])
        
        tmp = np.cumsum(experiments[experiment]['perc'], axis=1)
        tmp[:,:,0] = tmp[:,:,0] / tmp[:, :, 1]
               
        # perc.append(np.cumsum(experiments[experiment]['perc'], axis=1))
        perc.append(tmp[:,:,0])
        qds.append(experiments[experiment]['qds'])
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['feasible_coverage'],
                    title=f'{experiment}-feas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible_coverage'],
                    title=f'{experiment}-infeas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['errs'],
                    title=f'{experiment}-estimator-errs',
                    elites=False)
        print_stats(arr=tmp,
                    title=f'{experiment}-percentage-feas-increase',
                    elites=True)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=True,
                    overall=False)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-infeas-QD-score',
                    elites=False,
                    overall=False)

    colors = ['darkgreen', 'blue', 'purple']
    linestyles = ['dashed', 'dotted', 'dashdot']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=feas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average Feasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-feas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=infeas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average Infeasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-infeas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=errs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average estimator losses comparison ({N_RUNS} runs)',
                ylabel='Loss',
                partial_filename='avg-estimator-errs',
                plot_elites=True,
                add_area=False,
                legend_loc='upper right')
    
    plot_compare(arrs=perc,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'Average percentage feasible solution w/ infeasible parents ({N_RUNS} runs)',
                ylabel='Amount (%)',
                partial_filename='avg-percentage-feas-increase',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'QD-score (feasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-feas',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment05',
                title=f'QD-score (infeasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-infeas',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

## Experiment 6

Run variant FI-2Pop w/ scaled fitness (remember to set in configs!)

In [ ]:
def run_experiment(solver,
                   exp_name,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    i_fitnesses_hist = []
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            solver.reset()         
            f_pop, i_pop = solver.initialize(pops_size=pops_size,
                                             n_retries=n_retries)
            f_pop, i_pop = solver.fi2pop(f_pop=f_pop,
                                         i_pop=i_pop,
                                         n_iter=N_GENS)
            f_fitnesses_hist.append(solver.ffs)
            i_fitnesses_hist.append(solver.ifs)
            f_fitnesses = [cs.c_fitness for cs in f_pop]
            i_fitnesses = [cs.ncv for cs in f_pop]
            save_solution_stats(cs=f_pop[f_fitnesses.index(max(f_fitnesses))],
                                exp_name=exp_name,
                                dir_name='cog_experiment01')
            iterations.set_postfix(ordered_dict={'f_fit': np.max(f_fitnesses),
                                                 'i_fit': np.max(i_fitnesses) if isinstance(solver, FI2PopVariantSolver) else np.min(i_fitnesses)},
                                   refresh=True)
            
    return f_fitnesses_hist, i_fitnesses_hist


In [ ]:
if run_experiments[6]:
    solvers, exp_names = [], []

    for merge_method in [mean_merge, max_merge, min_merge]:
        buffer = Buffer(merge_method=merge_method)
        solver = FI2PopVariantSolver(feasible_fitnesses=feasible_fitnesses,
                                    lsystem=lsystem,
                                    buffer=buffer,
                                    estimator=MLPEstimator(len(feasible_fitnesses), 1))
        exp_name = f'variant-fi2pop-{buffer._merge.__name__}'
        
        solvers.append(solver)
        exp_names.append(exp_name)

    for solver, exp_name in zip(solvers, exp_names):
        f_fitnesses_hist, i_fitnesses_hist = run_experiment(solver=solver,
                                                            exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    exp_name=exp_name,
                    dir_name='cog_experiment06')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                exp_name=exp_name,
                                dir_name='cog_experiment06')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                    exp_name=exp_name,
                                    dir_name='cog_experiment06')


In [ ]:
if plot_experiments_results[6]:
    experiments = {
                'variant-fi2pop-max_merge': {},
                'variant-fi2pop-mean_merge': {},
                'variant-fi2pop-min_merge': {}
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment06/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']

    feas_arrs = []
    infeas_arrs = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        labels.append(experiment)
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        
    colors = ['blue', 'blue', 'blue']
    linestyles = ['dashed', 'dotted', 'dashdot']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment06',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment06',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment06',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment06',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

## Experiment 7

Run variant MAP-Elites with bandit agent.

In [ ]:
def run_experiment(mapelites: MAPElites,
                   exp_name: str,
                   pops_size: int = POP_SIZE,
                   n_retries: int = N_RETRIES):
    f_fitnesses_hist = []
    f_coverages_hist = []
    i_fitnesses_hist = []
    i_coverages_hist = []
    estimator_errors_hist = []
    percentage_new_feas_hist = []
    qd_scores_hist = []
    with open('estimators_perf.log', 'a') as f:
        f.write(f'Running experiment {exp_name}\n')
    with trange(N_RUNS, desc=f'Running experiments {exp_name}') as iterations:
        for n in iterations:
            with open('estimators_perf.log', 'a') as f:
                f.write(f'Run {n+1}/`{N_RUNS}\n')
            f_fitnesses, i_fitnesses = [], []
            f_coverages, i_coverages = [], []
            qd_scores, percentage_new_feas = [], []
            mapelites.reset(lcs=[])
            mapelites.hull_builder = None
            mapelites.generate_initial_populations(pops_size=pops_size,
                                                   n_retries=n_retries)
            f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
            i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
            qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
            fc_curr, total = mapelites.get_coverage(pop='feasible')
            ic_curr, _ = mapelites.get_coverage(pop='infeasible')
            f_coverages.append(fc_curr)
            i_coverages.append(ic_curr)
            with trange(N_GENS, desc=f'Running MAP-Elites (run {n+1})...') as gens:
                for gen in gens:
                    mapelites.emitter_step(gen=gen)
                    f_fitnesses.append(mapelites.get_fitness_metrics(pop='feasible'))
                    i_fitnesses.append(mapelites.get_fitness_metrics(pop='infeasible'))
                    fc_curr, total = mapelites.get_coverage(pop='feasible')
                    ic_curr, _ = mapelites.get_coverage(pop='infeasible')
                    f_coverages.append(fc_curr)
                    i_coverages.append(ic_curr)                    
                    if gen == N_GENS - 1:
                        f_coverages = [x / total for x in f_coverages]
                        i_coverages = [x / total for x in i_coverages]
                    
                    percentage_new_feas.append(mapelites.get_new_feas_with_unfeas_parents())
                                        
                    qd_scores.append((mapelites.get_qdscore(pop='feasible'), mapelites.get_qdscore(pop='infeasible')))
                    
                    
                    gens.set_postfix(ordered_dict={'f_top_fit': f_fitnesses[-1][0],
                                                   'f_avg_fit': f_fitnesses[-1][1],
                                                   'i_top_fit': i_fitnesses[-1][0],
                                                   'i_avg_fit': i_fitnesses[-1][1],
                                                   '%n': percentage_new_feas[-1][0] / percentage_new_feas[-1][1],
                                                   'f_qd': qd_scores[-1][0],
                                                   'i_qd': qd_scores[-1][1]},
                                     refresh=True)
                f_fitnesses_hist.append(f_fitnesses)
                i_fitnesses_hist.append(i_fitnesses)
                f_coverages_hist.append(f_coverages)
                i_coverages_hist.append(i_coverages)
                
                percentage_new_feas_hist.append(percentage_new_feas)
                qd_scores_hist.append(qd_scores)
                
                if mapelites.estimator:
                    missing = N_GENS - len(mapelites.estimator.train_losses)
                    if missing >= 0:
                        to_add = [np.nan for _ in range(missing)]
                        to_add.extend(mapelites.estimator.train_losses)
                    else:
                        to_add = mapelites.estimator.train_losses[:50]
                else:
                    to_add = [np.nan for _ in range(N_GENS)]
                estimator_errors_hist.append(to_add)
                
                
                save_solution_stats(cs=mapelites.get_random_elite(pop='feasible'),
                                    exp_name=f'{exp_name}-elites',
                                    dir_name='cog_experiment07')
                
                for i in range(mapelites.bins.shape[0]):
                    for j in range(mapelites.bins.shape[1]):
                        for cs in mapelites.bins[i, j]._feasible:
                            save_solution_stats(cs=cs,
                                                exp_name=exp_name,
                                                dir_name='cog_experiment07')
                
            iterations.set_postfix(ordered_dict={'f_coverage': f_coverages_hist[-1][-1],
                                                 'i_coverages': i_coverages_hist[-1][-1]})
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment07/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='fitness',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment07/{exp_name}-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='feasible',
                                  save_as=f'cog_experiment07/{exp_name}-coverage-run{str(n).zfill(2)}')
            mapelites.show_metric(metric='size',
                                  show_mean=True,
                                  population='infeasible',
                                  save_as=f'cog_experiment07/{exp_name}-coverage-run{str(n).zfill(2)}')
        return f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist


In [ ]:
if run_experiments[7]:
    l_mapelites, exp_names = [], []

    behavior_descriptors = (BehaviorCharacterization(name='mame',
                                                     func=mame,
                                                     bounds=(0, 20)),
                            BehaviorCharacterization(name='mami',
                                                     func=mami,
                                                     bounds=(0, 20)))
    bandit_actions = [
        'random-emitter;max',
        'random-emitter;median',
        'random-emitter;min',
        'optimising-emitter;max',
        'optimising-emitter;median',
        'optimising-emitter;min',
        'optimising-emitter-v2;max',
        'optimising-emitter-v2;median',
        'optimising-emitter-v2;min'
    ]
    mapelites = MAPElites(lsystem=lsystem,
                          feasible_fitnesses=feasible_fitnesses,
                          estimator=QuantileEstimator(xshape=len(feasible_fitnesses),
                                                      yshape=1),
                          buffer=Buffer(merge_method=mean_merge),
                          behavior_descriptors=behavior_descriptors,
                          n_bins=(32,32),
                          emitter=None,
                          agent=EpsilonGreedyAgent(bandits=[Bandit(action=x) for x in bandit_actions],
                                                   epsilon=0.1),
                          agent_rewards=[fitness_reward, coverage_reward])
    mapelites.allow_aging = False
    mapelites.allow_res_increase = False
    exp_name = f'variant-bandit-mapelites-{mapelites.buffer._merge.__name__}'
    l_mapelites.append(mapelites)
    exp_names.append(exp_name)

    for mapelites, exp_name in zip(l_mapelites, exp_names):
        f_fitnesses_hist, i_fitnesses_hist, f_coverages_hist, i_coverages_hist, estimator_errors_hist, percentage_new_feas_hist, qd_scores_hist = run_experiment(mapelites=mapelites,
                                                                                                                                                                    exp_name=exp_name)
        save_metrics(f_fitnesses_hist=f_fitnesses_hist,
                    i_fitnesses_hist=i_fitnesses_hist,
                    estimator_errs=estimator_errors_hist,
                    percentage_new_feas=percentage_new_feas_hist,
                    qd_scores=qd_scores_hist,      
                    exp_name=exp_name,
                    dir_name='cog_experiment07')
        save_coverage(f_coverages=f_coverages_hist,
                      i_coverages=i_coverages_hist,
                      exp_name=exp_name,
                      dir_name='cog_experiment07')
        plot_mean_and_top_feasible(ffs=f_fitnesses_hist,
                                   exp_name=exp_name,
                                   dir_name='cog_experiment07')
        plot_mean_and_top_infeasible(ifs=i_fitnesses_hist,
                                     exp_name=exp_name,
                                     dir_name='cog_experiment07')
        plot_coverage(coverages_hist=f_coverages_hist,
                      title=f'Avg. Feasible coverage',
                      exp_name=exp_name,
                      filename='avg-feas-coverage',
                      dir_name='cog_experiment07')
        plot_coverage(coverages_hist=i_coverages_hist,
                      title=f'Avg. Infeasible coverage',
                      exp_name=exp_name,
                      filename='avg-infeas-coverage',
                      dir_name='cog_experiment07')


In [ ]:
if plot_experiments_results[7]:
    experiments = {
                'variant-bandit-mapelites-mean_merge': {},
    }

    for experiment in experiments.keys():
        with open(f'results/cog_experiment07/{experiment}_metrics.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible'] = npzfile['arr_0']
            experiments[experiment]['infeasible'] = npzfile['arr_1']
        with open(f'results/cog_experiment07/{experiment}_coverages.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
            experiments[experiment]['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
    for experiment in experiments.keys():
        with open(f'results/cog_experiment07/{experiment}_estimator_errs.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['errs'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment07/{experiment}_percentage_new_feas.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['perc'] = npzfile['arr_0']
    for experiment in experiments.keys():
        with open(f'results/cog_experiment07/{experiment}_qds.npz', 'rb') as f:
            npzfile = np.load(f)
            experiments[experiment]['qds'] = npzfile['arr_0']
            
    feas_arrs = []
    infeas_arrs = []
    feas_coverages = []
    infeas_coverages = []
    errs = []
    perc = []
    qds = []
    labels = []
    for experiment in experiments.keys():
        feas_arrs.append(experiments[experiment]['feasible'])
        infeas_arrs.append(experiments[experiment]['infeasible'])
        feas_coverages.append(experiments[experiment]['feasible_coverage'])
        infeas_coverages.append(experiments[experiment]['infeasible_coverage'])
        errs.append(experiments[experiment]['errs'])
        
        tmp = np.cumsum(experiments[experiment]['perc'], axis=1)
        tmp[:,:,0] = tmp[:,:,0] / tmp[:, :, 1]
               
        # perc.append(np.cumsum(experiments[experiment]['perc'], axis=1))
        perc.append(tmp[:,:,0])
        qds.append(experiments[experiment]['qds'])
        labels.append(experiment)
        
        
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-top-feas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['feasible'],
                    title=f'{experiment}-avg-feas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['feasible_coverage'],
                    title=f'{experiment}-feas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-top-infeas-fitness',
                    elites=True)
        print_stats(arr=experiments[experiment]['infeasible'],
                    title=f'{experiment}-avg-infeas-fitness',
                    elites=False)
        print_stats(arr=experiments[experiment]['infeasible_coverage'],
                    title=f'{experiment}-infeas-coverage',
                    elites=True)
        print_stats(arr=experiments[experiment]['errs'],
                    title=f'{experiment}-estimator-errs',
                    elites=False)
        # print_stats(arr=experiments[experiment]['perc'],
        print_stats(arr=tmp,
                    title=f'{experiment}-percentage-feas-increase',
                    elites=True)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=True,
                    overall=False)
        print_stats(arr=experiments[experiment]['qds'],
                    title=f'{experiment}-feas-QD-score',
                    elites=False,
                    overall=False)

    colors = ['blue']
    linestyles = ['dashed']

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Elite feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-feas-fitness',
                plot_elites=True,
                add_area=True)

    plot_compare(arrs=feas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average feasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-feas-fitness',
                plot_elites=False,
                add_area=True)

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Top infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='top-infeas-fitness',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')

    plot_compare(arrs=infeas_arrs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average infeasible fitnesses comparison ({N_RUNS} runs)',
                ylabel='Fitness',
                partial_filename='avg-infeas-fitness',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=feas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average Feasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-feas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=infeas_coverages,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average Infeasible coverage comparison ({N_RUNS} runs)',
                ylabel='Coverage',
                partial_filename='avg-infeas-coverage',
                plot_elites=True,
                add_area=True,
                legend_loc='lower right')
    
    plot_compare(arrs=errs,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average estimator losses comparison ({N_RUNS} runs)',
                ylabel='Loss',
                partial_filename='avg-estimator-errs',
                plot_elites=True,
                add_area=False,
                legend_loc='upper right')
    
    plot_compare(arrs=perc,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'Average percentage feasible solution w/ infeasible parents ({N_RUNS} runs)',
                ylabel='Amount (%)',
                partial_filename='avg-percentage-feas-increase',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'QD-score (feasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-feas',
                plot_elites=True,
                add_area=True,
                legend_loc='upper right')
    plot_compare(arrs=qds,
                colors=colors,
                linestyles=linestyles,
                labels=labels,
                dir_name='cog_experiment07',
                title=f'QD-score (infeasible) ({N_RUNS} runs)',
                ylabel='Score',
                partial_filename='qdscore-infeas',
                plot_elites=False,
                add_area=True,
                legend_loc='upper right')

# Final Plots

## FI-2Pop

In [ ]:
experiments = {
               'variant-fi2pop-max_merge': {},
               'variant-fi2pop-mean_merge': {},
               'variant-fi2pop-min_merge': {}
               }

for experiment in experiments.keys():
    with open(f'results/cog_experiment01/{experiment}_metrics.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments[experiment]['feasible'] = npzfile['arr_0']
        experiments[experiment]['infeasible'] = npzfile['arr_1']
    with open(f'results/cog_experiment01/{experiment}_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments[experiment]['perc'] = npzfile['arr_0']

experiments['standard-fi2pop'] = {}
with open(f'results/cog_experiment01/standard-fi2pop_metrics.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['standard-fi2pop']['feasible'] = npzfile['arr_0']
        experiments['standard-fi2pop']['infeasible'] = npzfile['arr_1']
with open(f'results/cog_experiment01/standard-fi2pop_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['standard-fi2pop']['perc'] = npzfile['arr_0']


feas_arrs = []
infeas_arrs = []
perc = []
labels = ['M-FI-2Pop', 'μ-FI-2Pop', 'm-FI-2Pop', 'FI-2-Pop']
for experiment in experiments.keys():
    feas_arrs.append(experiments[experiment]['feasible'])
    infeas_arrs.append(experiments[experiment]['infeasible'])
    perc.append(np.cumsum(experiments[experiment]['perc'], axis=1))
    
colors = ['blue', 'purple', 'red', 'darkgreen']
linestyles = ['dashed', 'dotted', 'dashdot', 'solid']

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-top-feas-fitness-std',
            plot_elites=True,
            add_area=True)

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-top-feas-fitness-se',
            plot_elites=True,
            add_area=True,
            use_se=True)

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-top-feas-fitness-ci',
            plot_elites=True,
            add_area=True,
            use_ci=True)

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-avg-feas-fitness-std',
            plot_elites=False,
            add_area=True)

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-avg-feas-fitness-se',
            plot_elites=False,
            add_area=True,
            use_se=True)

plot_compare(arrs=feas_arrs,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='Fitness',
            partial_filename='fi2pop-avg-feas-fitness-ci',
            plot_elites=False,
            add_area=True,
            use_ci=True)

plot_compare(arrs=perc,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='%',
            partial_filename='fi2pop-perc-feas-increase-std',
            plot_elites=False,
            add_area=True)

plot_compare(arrs=perc,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='%',
            partial_filename='fi2pop-perc-feas-increase-se',
            plot_elites=False,
            add_area=True,
            use_se=True)

plot_compare(arrs=perc,
            colors=colors,
            linestyles=linestyles,
            labels=labels,
            dir_name='.',
            title=None,
            ylabel='%',
            partial_filename='fi2pop-perc-feas-increase-ci',
            plot_elites=False,
            add_area=True,
            use_ci=True)

## MAP-Elites

In [ ]:
experiments = {}

experiments['standard-mapelites'] = {}
with open(f'results/cog_experiment02/standard-mapelites_metrics.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['standard-mapelites']['feasible'] = npzfile['arr_0']
with open(f'results/cog_experiment02/standard-mapelites_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['standard-mapelites']['feasible_coverage'] = np.expand_dims(
        npzfile['arr_0'], 2)
with open(f'results/cog_experiment02/standard-mapelites_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['standard-mapelites']['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
with open(f'results/cog_experiment02/standard-mapelites_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['standard-mapelites']['perc'] = npzfile['arr_0']
with open(f'results/cog_experiment02/standard-mapelites_qds.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['standard-mapelites']['qds'] = npzfile['arr_0']

experiments['variant-mapelites-mean-merge'] = {}
with open(f'results/cog_experiment02/variant-mapelites-mean_merge_metrics.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-mapelites-mean-merge']['feasible'] = npzfile['arr_0']
with open(f'results/cog_experiment02/variant-mapelites-mean_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-mapelites-mean-merge']['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
with open(f'results/cog_experiment02/variant-mapelites-mean_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-mapelites-mean-merge']['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
with open(f'results/cog_experiment02/variant-mapelites-mean_merge_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['variant-mapelites-mean-merge']['perc'] = npzfile['arr_0']
with open(f'results/cog_experiment02/variant-mapelites-mean_merge_qds.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-mapelites-mean-merge']['qds'] = npzfile['arr_0']

experiments['variant-optim2-mapelites-min-merge'] = {}
with open(f'results/cog_experiment05/variant-optim-mapelites-min_merge_metrics.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-optim2-mapelites-min-merge']['feasible'] = npzfile['arr_0']
with open(f'results/cog_experiment05/variant-optim-mapelites-min_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-optim2-mapelites-min-merge']['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
with open(f'results/cog_experiment05/variant-optim-mapelites-min_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-optim2-mapelites-min-merge']['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
with open(f'results/cog_experiment05/variant-optim-mapelites-min_merge_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['variant-optim2-mapelites-min-merge']['perc'] = npzfile['arr_0']
with open(f'results/cog_experiment05/variant-optim-mapelites-min_merge_qds.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-optim2-mapelites-min-merge']['qds'] = npzfile['arr_0']

experiments['variant-bandit-mapelites-mean-merge'] = {}
with open(f'results/cog_experiment07/variant-bandit-mapelites-mean_merge_metrics.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-bandit-mapelites-mean-merge']['feasible'] = npzfile['arr_0']
with open(f'results/cog_experiment07/variant-bandit-mapelites-mean_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-bandit-mapelites-mean-merge']['feasible_coverage'] = np.expand_dims(npzfile['arr_0'], 2)
with open(f'results/cog_experiment07/variant-bandit-mapelites-mean_merge_coverages.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-bandit-mapelites-mean-merge']['infeasible_coverage'] = np.expand_dims(npzfile['arr_1'], 2)
with open(f'results/cog_experiment07/variant-bandit-mapelites-mean_merge_percentage_new_feas.npz', 'rb') as f:
        npzfile = np.load(f)
        experiments['variant-bandit-mapelites-mean-merge']['perc'] = npzfile['arr_0']
with open(f'results/cog_experiment07/variant-bandit-mapelites-mean_merge_qds.npz', 'rb') as f:
    npzfile = np.load(f)
    experiments['variant-bandit-mapelites-mean-merge']['qds'] = npzfile['arr_0']

feas_arrs = []
feas_coverages = []
infeas_coverages = []
perc = []
qds = []
for experiment in experiments.keys():
    feas_arrs.append(experiments[experiment]['feasible'])
    feas_coverages.append(experiments[experiment]['feasible_coverage'])
    infeas_coverages.append(experiments[experiment]['infeasible_coverage'])
    tmp = np.cumsum(experiments[experiment]['perc'], axis=1)
    tmp[:,:,0] = tmp[:,:,0] / tmp[:, :, 1]
    perc.append(tmp[:,:,0])
    qds.append(experiments[experiment]['qds'])

labels = ['CMAP-Elites', 'μ-CMAP-Elites', 'Em-CMAP-Elites', 'B-CMAP-Elites']

colors = ['darkgreen', 'blue', 'purple', 'red']
linestyles = ['solid', 'dashed', 'dotted', 'dashdot']

plot_compare(arrs=feas_arrs,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Fitness',
             partial_filename='mapelites-top-feas-fitness-std',
             plot_elites=True,
             add_area=True)

plot_compare(arrs=feas_arrs,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Fitness',
             partial_filename='mapelites-top-feas-fitness-se',
             plot_elites=True,
             add_area=True,
             use_se=True)

plot_compare(arrs=feas_arrs,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Fitness',
             partial_filename='mapelites-avg-feas-fitness-std',
             plot_elites=False,
             add_area=True,
            legend_loc='upper left')

plot_compare(arrs=feas_arrs,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Fitness',
             partial_filename='mapelites-avg-feas-fitness-se',
             plot_elites=False,
             add_area=True,
             legend_loc='upper left',
             use_se=True)

plot_compare(arrs=feas_coverages,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Coverage',
             partial_filename='mapelites-avg-feas-coverage-std',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left')

plot_compare(arrs=feas_coverages,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Coverage',
             partial_filename='mapelites-avg-feas-coverage-se',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left',
             use_se=True)

plot_compare(arrs=infeas_coverages,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Coverage',
             partial_filename='mapelites-avg-infeas-coverage-std',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left')

plot_compare(arrs=infeas_coverages,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Coverage',
             partial_filename='mapelites-avg-infeas-coverage-se',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left',
             use_se=True)

plot_compare(arrs=perc,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Amount (%)',
             partial_filename='mapelites-perc-feas-increase-std',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left')

plot_compare(arrs=perc,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='Amount (%)',
             partial_filename='mapelites-perc-feas-increase-se',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left',
             use_se=True)

plot_compare(arrs=qds,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='QD-Score',
             partial_filename='mapelites-feas-qdscores-std',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left')

plot_compare(arrs=qds,
             colors=colors,
             linestyles=linestyles,
             labels=labels,
             dir_name='.',
             title=None,
             ylabel='QD-Score',
             partial_filename='mapelites-feas-qdscores-se',
             plot_elites=True,
             add_area=True,
             legend_loc='upper left',
             use_se=True)
